In [17]:
# First we load autoload, so we don't need to keep restarting the kernel to get
# new definitions. This ensures that functions are reloaded from the file whenever
# the file is changed.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# All the stuff we need
from lark import *
from lark.tree import Tree
from lark.lexer import Token
from ast import literal_eval
import numpy as np

from helpers import *
from show import *
from type import *
from PEjames import *

In [19]:
#
# The following reads in a full program and parses it into an AST `program_tree`.
#
cqparse = Lark.open("CQ.lark",parser='lalr', start="program")
program = read_file("../CQ-programs/qft2.cq")

# Remove comment to test with `initialize2.cq`
#program = read_file("../CQ-programs/initialize2.cq") 
program_tree=prune_tree(cqparse.parse(program, start="program"))
# If we've parsed the program properly, we can translate it back into CQ syntax
#print(show_program(program_tree))

#print(ast_to_yaml(program_tree))
#print(ast_to_yaml(prune_tree(program_tree)))

print(show_program(program_tree))
print(program_tree.pretty())

qft(qbit q[d]) {
  int i = 0 ;

  while((i < (d - 1)))
  {
    int j = (i + 1) ;
    int angle = 2 ;

    H q[i] ;
    H q[(i + 1)] ;
    while((j < d))
    {
      Rz(((2 * pi) / angle)) q[i] if q[j]) ;
      angle = (angle * 2) ;
      j = (j + 1) ;
    }

    i = (i + 1) ;
  }

}

program
  procedure
    qft
    parameter_declarations
      parameter_declaration
        qbit
        q
        d
    statement
      block
        declarations
          declaration
            int
            i
            exp
              exp	0
        statements
          statement
            while
            exp
              exp
                exp
                  lval	i
              <
              exp
                exp
                  lval	d
                -
                exp	1
            statement
              block
                declarations
                  declaration
                    int
                    j
                    exp
                      exp
            

In [20]:
# It's easiest to test your program bit by bit. 
# We use the start symbol to get parsers for individual syntactical elements.
# The following are the parsers for expressions, declarations, and statements.
exp_parser  = Lark.open("CQ.lark", start="exp")
decl_parser = Lark.open("CQ.lark", start="declaration")
stat_parser = Lark.open("CQ.lark", start="statement")

# Some test examples in order of increasing difficulty
# Make sure that the variables you want to be statically evaluated
# are defined in your static_input dictionary below.
# We will use the ASTs to test the type checker and the PE further below.
# If you get an error already here, then there is an issue with your parser.
# Change these up to experiment.
t = parse_and_prune(exp_parser,"2*arccos(sqrt(a[0]*a[0] + a[2]*a[2]))")
print(ast_to_yaml(t))
d = parse_and_prune(decl_parser,"float th;")
d = parse_and_prune(decl_parser,"float th = 2*arccos(sqrt(a[0]*a[0] + a[2]*a[2]));")
d = parse_and_prune(decl_parser,"float x[4] = { 1/sqrt(2), 1/sqrt(4), 1/sqrt(6), 1/sqrt(12) };")
s = parse_and_prune(stat_parser,"a[0] = 2*arccos(sqrt(a[0]*a[0] + a[2]*a[2]));")
s = parse_and_prune(stat_parser,"if (2+2==4) a[0] = 1; else a[1] = 1;") 
#s = parse_and_prune(stat_parser,"if (2+2 < (5*a[1])) a[1] = 1; else a[0] = 1;")
#s = parse_and_prune(stat_parser,"{}")
#s = parse_and_prune(stat_parser,"while (i<2){ int j = a[i]; a[i] = a[3-i]; a[3-i] = j; i=i+1; }")
#s = parse_and_prune(stat_parser,"while (i<2){ int j = b[i]; b[i] = b[3-i]; b[3-i] = j; i=i+1; }")
#s = parse_and_prune(stat_parser,"{ a[i] = i+1; i = a[i]; }")
s = parse_and_prune(stat_parser,"{ int d = 10; { int a[d]; int i; int j=1; i = 0; while (i < d-1) { a[i+1] = j; i = i + 1; } } }")
initialize_stat="""{
    float a[4] = {1/sqrt(2), 1/sqrt(4), 1/sqrt(6), 1/sqrt(12)};
    qbit  q[2];
    float th1 = 2*arccos(sqrt(a[0]*a[0] + a[2]*a[2]));
    float th2 = 2*arctan2(a[3],a[1]); 
    float th3 = 2*arctan2(a[2],a[0]);

    Ry(th1) q[0];
    Ry(th2) q[1] if q[0];
    not q[0];
    Ry(th3) q[1] if q[0];
    not q[0];
}
"""
#s = parse_and_prune(stat_parser,initialize_stat)

   - tree: exp
     children: # 1 children, ['exp']
     - tree: exp
       children: # 3 children, ['exp', 'BINOP', 'exp']
       - tree: exp
         children: # 1 children, [Token('TERMINAL', 'INT')]
         - token: INT
           value: '2'
       - token: BINOP
         value: '*'
       - tree: exp
         children: # 2 children, [Token('TERMINAL', 'BUILTIN_FUN1'), 'exp']
         - token: BUILTIN_FUN1
           value: 'arccos'
         - tree: exp
           children: # 1 children, ['exp']
           - tree: exp
             children: # 2 children, [Token('TERMINAL', 'BUILTIN_FUN1'), 'exp']
             - token: BUILTIN_FUN1
               value: 'sqrt'
             - tree: exp
               children: # 1 children, ['exp']
               - tree: exp
                 children: # 3 children, ['exp', 'BINOP', 'exp']
                 - tree: exp
                   children: # 3 children, ['exp', 'BINOP', 'exp']
                   - tree: exp
                     children: # 1 c

In [21]:
# Test of the type checker on the exp, declaration, and statement ASTs.
type_env0=[{},{'a': "float[4]", 'b':'int[4]', "i": "int"}]

# Test type checker on smaller syntactical elements:
print(type_exp(t,type_env0))
print(type_declaration(d,type_env0))
#print(show_declaration(d))
#print(show_statement(s))
print(type_statement(s,type_env0))


float
('x', 'float[4]')
True


In [22]:
# Type check the entire program
type_program(program_tree)

Procedure qft is well-typed


True

In [23]:
type_env0=[{},{'a': "float[4]"}]
static_input = {'a': np.sqrt([1/2.,1/4.,1/6., 1/12.]),'d':4}
value_env0=[static_input]
#type_env0=[{},{'a': "int[4]", 'i': "int"}]
#value_env0=[{'a': np.array([4,3,2,1]), 'i':0}]


#Test PE for a declaration AST:
(ped,static) = PE_declaration(d,value_env0)
if not static:
    print(show_declaration(ped))
else: 
    print(f"`{show_declaration(d)}` fully evaluated to {show_declaration(ped)}, environment is:\n {value_env0}")


`float x[4] = [(1 / sqrt(2)),(1 / sqrt(4)),(1 / sqrt(6)),(1 / sqrt(12))] ;` fully evaluated to float x[4] = [0.7071067811865475,0.5,0.4082482904638631,0.2886751345948129] ;, environment is:
 [{'a': array([0.70710678, 0.5       , 0.40824829, 0.28867513]), 'd': 4, 'x': [0.7071067811865475, 0.5, 0.4082482904638631, 0.2886751345948129]}]


In [24]:

# Test PE for a statement AST:
(pes,static) = PE_statement(s,value_env0)
if not static:
    #print(f"residual statement:\n\t{pes}\noriginal statement:\n\t{s}")
    #print(pes.pretty())
    print(show_statement(pes))
    print(value_env0)
else: 
    print(f"{show_statement(s)} fully evaluated,\n environment is {value_env0}")    
    print(value_env0)

{
  {
    int i ;

    i = 0 ;
    while((i < 9))
    {
      a[(i + 1)] = j ;
      i = (i + 1) ;
    }

  }

}

[{'a': array([0.70710678, 0.5       , 0.40824829, 0.28867513]), 'd': 10, 'x': [0.7071067811865475, 0.5, 0.4082482904638631, 0.2886751345948129]}]


In [25]:
# Test PE for a full program (Once you get this far!):
pt_res = PE_program(program_tree,static_input) 

In [26]:
print(show_program(pt_res))

qft(qbit q[10]) {
  {
    {
      H q[0] ;
      H q[1] ;
      {
        {
          Rz(3.141592653589793) q[0] if q[1]) ;
        }

        {
          Rz(1.5707963267948966) q[0] if q[2]) ;
        }

        {
          Rz(0.7853981633974483) q[0] if q[3]) ;
        }

        {
          Rz(0.39269908169872414) q[0] if q[4]) ;
        }

        {
          Rz(0.19634954084936207) q[0] if q[5]) ;
        }

        {
          Rz(0.09817477042468103) q[0] if q[6]) ;
        }

        {
          Rz(0.04908738521234052) q[0] if q[7]) ;
        }

        {
          Rz(0.02454369260617026) q[0] if q[8]) ;
        }

        {
          Rz(0.01227184630308513) q[0] if q[9]) ;
        }

      }

    }

    {
      H q[1] ;
      H q[2] ;
      {
        {
          Rz(3.141592653589793) q[1] if q[2]) ;
        }

        {
          Rz(1.5707963267948966) q[1] if q[3]) ;
        }

        {
          Rz(0.7853981633974483) q[1] if q[4]) ;
        }

        {
          Rz(0.39269

In [27]:
from flatten import *

print(show_program(flatten_program(pt_res)))


qft(qbit q[10]) {
  H q[0] ;
  H q[1] ;
  Rz(3.141592653589793) q[0] if q[1]) ;
  Rz(1.5707963267948966) q[0] if q[2]) ;
  Rz(0.7853981633974483) q[0] if q[3]) ;
  Rz(0.39269908169872414) q[0] if q[4]) ;
  Rz(0.19634954084936207) q[0] if q[5]) ;
  Rz(0.09817477042468103) q[0] if q[6]) ;
  Rz(0.04908738521234052) q[0] if q[7]) ;
  Rz(0.02454369260617026) q[0] if q[8]) ;
  Rz(0.01227184630308513) q[0] if q[9]) ;
  H q[1] ;
  H q[2] ;
  Rz(3.141592653589793) q[1] if q[2]) ;
  Rz(1.5707963267948966) q[1] if q[3]) ;
  Rz(0.7853981633974483) q[1] if q[4]) ;
  Rz(0.39269908169872414) q[1] if q[5]) ;
  Rz(0.19634954084936207) q[1] if q[6]) ;
  Rz(0.09817477042468103) q[1] if q[7]) ;
  Rz(0.04908738521234052) q[1] if q[8]) ;
  Rz(0.02454369260617026) q[1] if q[9]) ;
  H q[2] ;
  H q[3] ;
  Rz(3.141592653589793) q[2] if q[3]) ;
  Rz(1.5707963267948966) q[2] if q[4]) ;
  Rz(0.7853981633974483) q[2] if q[5]) ;
  Rz(0.39269908169872414) q[2] if q[6]) ;
  Rz(0.19634954084936207) q[2] if q[7]) ;
  Rz

In [28]:
t = parse_and_prune(exp_parser,"1+1+var")
PE_exp(t,[{'var':2 }])

(Tree(Token('RULE', 'exp'), [Token('INT', '4')]), True)

In [29]:
from lark import Lark
from PE_skeleton import PE_exp
from helpers import parse_and_prune
from show import show_exp

exp_parser  = Lark.open("CQ.lark", start="exp")

e = parse_and_prune(exp_parser,"1+1+a+b")

(ped,static) = PE_exp(e,[{'a':2}])

print(f"{show_exp(e)} evaluated to {show_exp(ped)} (AST {ped})")

(((1 + 1) + a) + b) evaluated to (4 + b) (AST Tree(Token('RULE', 'exp'), [Tree(Token('RULE', 'exp'), [Token('INT', '4')]), Token('BINOP', '+'), Tree(Token('RULE', 'exp'), [Tree(Token('RULE', 'lval'), [Token(Token('TERMINAL', 'ID'), 'b')])])]))
